In [1]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')

In [2]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .config("spark.driver.memory", "16G")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cd2d7a30-f3d2-44d6-b1d5-931584e74a60;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [4]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType
from dateutil import parser
from pyspark.sql import DataFrame as SparkDataFrame,DataFrame
from datetime import datetime
from typing import List
from dateutil import parser
import json
import pyspark.sql.functions as f
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, BinaryType, IntegerType, TimestampType
from pyspark.sql import Column
from pyspark.sql import DataFrame as SparkDataFrame,DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from delta.tables import DeltaTable
from datetime import datetime, timedelta


In [5]:

def generete_symlink_manifest(spark:SparkSession, bucket:str) -> None:
        """
        Gera o arquivo symlink manifest para a tabela Delta do bucket de referencia
        """
        delta_table = DeltaTable.forPath(spark, bucket)
        delta_table.generate("symlink_format_manifest")


def path_exists(path, spark:SparkSession):
        """
        Dado um caminho verifica se existe ou não
        """
        sc = spark.sparkContext
        fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
            sc._jvm.java.net.URI.create("gs://" + path.split("/")[2]),
            sc._jsc.hadoopConfiguration(),
        )
        return fs.exists(sc._jvm.org.apache.hadoop.fs.Path(path))


def last_date(delta_path:str, spark:SparkSession):
        """
        Retorna a data (timestamp_kafka) do último registro da tabela Delta
        """
        if not path_exists(delta_path + 'job_control/last_date', spark):
            try:
                df_temp = (
                    spark
                    .read
                    .format("delta")
                    .load(delta_path)
                )

                list_result = (
                    df_temp
                    .agg({"timestamp_kafka": "max"}).collect()
                )

                last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
                snapshot_flag = False
            except:
                snapshot_flag = True
                last_date = ('01-01-2021 12:00AM')

            df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
            df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')
        else:
            snapshot_flag = False
            df = spark.read.format("csv").load(delta_path + 'job_control/last_date', header = 'true')
            last_date = df.collect()[0].asDict()['last_date']

        return parser.parse(last_date),snapshot_flag


def update_last_date(delta_path:str, df:DataFrame, spark:SparkSession):
        """
        Atualiza a data (timestamp_kafka) do último registro da tabela Delta
        """
        list_result = (
            df
            .agg({"timestamp_kafka": "max"}).collect()
        )

        last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
        df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
        df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')

        return last_date

def get_operations_counts(df:DataFrame, spark:SparkSession):
        """
        Retorna o número de registros de cada operação kafka (__op)
        """
        op_count_list =(
            df
            .groupBy('__op')
            .count()
            .collect()
        )
        
        new_data_count = 0
        updating_data_count = 0
        deleting_data_count = 0

        if len(op_count_list) > 0:
            for i in op_count_list:
                dic_temp = i.asDict()
                if (dic_temp['__op'] == 'r'):
                    new_data_count = new_data_count + dic_temp['count']
                elif (dic_temp['__op'] != 'r'):
                    updating_data_count = updating_data_count + dic_temp['count']
                elif (dic_temp['__op'] == 'd'):
                    deleting_data_count = deleting_data_count + dic_temp['count']

        new_total_data_count = new_data_count + updating_data_count + deleting_data_count
        
        return new_total_data_count, new_data_count, updating_data_count, deleting_data_count


def get_data(spark:SparkSession, bucket:str, date_colunm_name:str, date_begin:datetime, source_format:str='delta', is_landing:bool=False):
        """
        Carrega Dataframe filtrando os dados do bucket de referência conforme 
        data de início enviada no parametro

        :param str date_begin: Data de início para filtro dos dados
        """
        if is_landing:
            schema = StructType() \
                    .add("key",BinaryType(),True) \
                    .add("value",BinaryType(),True) \
                    .add("topic",StringType(),True) \
                    .add("partition",IntegerType(),True) \
                    .add("offset",StringType(),True) \
                    .add("timestamp",TimestampType(),True)
            df = (
                spark
                .read
                .format(source_format)
                .schema(schema)
                .load(bucket)
                .where(
                    (f.col("year")>date_begin.year)|
                    (
                        (f.col("year")==date_begin.year)&
                        (
                        (f.col("month")>date_begin.month)|
                        (
                            (f.col("month")==date_begin.month)&
                            (
                            (f.col("day")>date_begin.day)|
                            (
                                (f.col("day")==date_begin.day)&
                                (f.col("hour")>=date_begin.hour)&
                                (f.col(date_colunm_name)>date_begin)
                            )
                            )
                        )
                        )          
                    ) 
                )
            )
        else:
            df = (
                spark
                .read
                .format(source_format)
                .load(bucket)
                .where(
                    (f.col("year")>date_begin.year)|
                    (
                        (f.col("year")==date_begin.year)&
                        (
                        (f.col("month")>date_begin.month)|
                        (
                            (f.col("month")==date_begin.month)&
                            (
                            (f.col("day")>date_begin.day)|
                            (
                                (f.col("day")==date_begin.day)&
                                (f.col("hour")>=date_begin.hour)&
                                (f.col(date_colunm_name)>date_begin)
                            )
                            )
                        )
                        )          
                    ) 
                )
            )
        return df


def write_new_data(spark:SparkSession, df:DataFrame, bucket:str, destination_format:str='delta'):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'r'
        """
        df_r = df.where((f.col('__op') == 'r'))
        (
            df_r
            .write
            .format(destination_format)
            .mode("append")
            .option("checkpointLocation", bucket + "checkpoint")
            .save(bucket)
        )
        
        return True


def update_new_data(spark:SparkSession, df:DataFrame, bucket:str, pk:list=['pk_single']):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'c' e
        atualiza os dados existentes, cuja a operação kafka é 'u' ou 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        df_table_u = df.where((f.col('__op') != 'r'))
        df_table_u_unique = df_table_u.orderBy('timestamp_kafka', ascending = False).dropDuplicates(pk)
        merge_string = 'destination.' + pk[0] + ' = updates.' + pk[0]
        (
            table.alias('destination')
            .merge(df_table_u_unique.alias('updates'), merge_string)
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )
        
        return True


def delete_new_data(spark:SparkSession, bucket:str):
        """
        Deleta da tabela delta, no bucket de destino, os dados cuja a operação kafka é 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        table.delete(f.col('__op') == 'd')
        
        return True


def flatten_df(nested_df):
        """
        Retorna um dataframe com as colunas de um dataframe aninhado
        """
        flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
        nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

        flat_df = nested_df.select(flat_cols +
                                [f.col(nc+'.'+c)
                                    for nc in nested_cols
                                    for c in nested_df.select(nc+'.*').columns])
        return flat_df


def write_bronze_table(df:DataFrame, bucket:str):
        """
        Escreve no bucket de destino a tabela delta bronze
        """
        (
            df
            .write
            .partitionBy("year", "month", "day", "hour")
            .format("delta")
            .mode("append")
            .option("checkpointLocation", bucket + "checkpoint")
            .save(bucket)
        )

        return True

def get_schema(df:DataFrame):
    json_str = df.limit(1).toPandas()['fixedValue'][0]
    json_fields = json.loads(json_str)['schema']['fields']
    struct_fields = []
    for field in json_fields:
        struct_fields.append(StructField(field['field'], StringType()))
        
    return StructType(struct_fields)

def decode_data_frame(df:DataFrame, schema:StructType):
    df_decoded = ( 
        df
        .withColumn('fixedValue', f.col("value").cast("string"))
        .select(f.get_json_object('fixedValue',"$.payload").alias('payload'), 'timestamp_kafka')
        .withColumnRenamed('timestamp', 'timestamp_kafka')
        .select(f.from_json(f.col('payload'), schema).alias("DF"), 'timestamp_kafka')
        .select("DF.*" , 'timestamp_kafka')
    )
    
    return df_decoded

In [6]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [7]:

#PATH SILVER ZONE
SILVER_PATH_VENDA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/venda/'
SILVER_PATH_VENDA_ITEM = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/venda_item/'
SILVER_PATH_FILIAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/filial/'
SILVER_PATH_REDE_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/rede_loja/'
SILVER_PATH_EMPRESA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/empresa/'
SILVER_PATH_PRODUTO_BARRA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/mb_produtos_barra/'
SILVER_PATH_VENDEDOR_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/vendedor_loja/'
SILVER_PATH_FILIAL_PONTO = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/filial_ponto/'
SILVER_PATH_SITE = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/site/'
SILVER_PATH_GERENTE_REGIONAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/gerente_regional/'
SILVER_PATH_CONSULTOR = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/consultor/'

SILVER_PATH_PRODUTOS =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produtos/'
SILVER_PATH_PRODUTOS_BARRA_LINX = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produtos_barra/'
SILVER_PATH_HR_WEB_CATALOGO_SKU =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_sku/'
SILVER_PATH_HR_WEB_CATALOGO_FOTO =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_foto/'
SILVER_PATH_PRODUTO_CORES =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produto_cores/'

SILVER_PATH_BASE_CLIENTES =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher361/db_analise/crm/tb_base_clientes/'

GOLD_PATH_STORES = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/view/stores/'

In [8]:
venda  = DeltaTable.forPath(spark, SILVER_PATH_VENDA)
venda_item  = DeltaTable.forPath(spark, SILVER_PATH_VENDA_ITEM)
filial = DeltaTable.forPath(spark, SILVER_PATH_FILIAL)
rede_loja = DeltaTable.forPath(spark, SILVER_PATH_REDE_LOJA)
empresa = DeltaTable.forPath(spark, SILVER_PATH_EMPRESA)
stores = DeltaTable.forPath(spark, GOLD_PATH_STORES)

In [9]:
# Creating or replace TempView
venda.toDF().createOrReplaceTempView("view_venda")
filial.toDF().createOrReplaceTempView("view_filial")
rede_loja.toDF().createOrReplaceTempView("view_rede_loja")
empresa.toDF().createOrReplaceTempView("view_empresa")
stores.toDF().createOrReplaceTempView("view_stores")

In [10]:
sql =  '''
(SELECT
    IFNULL(sha2(CASE WHEN (length(REPLACE(REPLACE(VENDA.CPF_CNPJ_CLIENTE,'.',''),'-','')) NOT IN (11,14)) THEN '000000000000000' ELSE VENDA.CPF_CNPJ_CLIENTE END,256),'') AS CUSTOMER_ID
    ,from_unixtime(unix_timestamp( VENDA.DATA),'yyyy-MM-dd') AS ORDER_DATE
    ,CONCAT(RIGHT(CONCAT('00000', CAST(TRIM(FILIAL.ID_FILIAL) AS STRING)),6)
            ,REPLACE(CAST(CAST(VENDA.DATA AS DATE) AS STRING), '-', '')
            ,CAST(TRIM(VENDA.TICKET) AS STRING))
    AS ORDER_ID
    ,TRIM(FILIAL.CODIGO_FILIAL) AS ORDER_STORE
    ,CASE WHEN FILIAL.CODIGO_FILIAL ='84' THEN 'E' ELSE 'L' END AS ORDER_STORE_TYPE
    ,'' AS ORDER_DELIVERY_TYPE
    ,'' AS ORDER_STATUS
    ,'' AS ORDER_PAYMENT_METHOD
    ,'' AS ORDER_PAYMENT_TYPE
    ,'' AS ORDER_INSTALLMENTS
    ,VENDA.VALOR_VENDA AS ORDER_TOTAL
    ,VENDA.__op
    ,VENDA.timestamp_kafka
    ,'HERING KIDS' AS MARCA
    FROM  view_venda AS VENDA
    INNER JOIN view_filial AS FILIAL ON FILIAL.ID_FILIAL=VENDA.ID_FILIAL
    INNER JOIN view_rede_loja AS REDE_LOJA   ON REDE_LOJA.ID_REDE_LOJA =FILIAL.ID_REDE_LOJA
    INNER JOIN view_empresa AS EMPRESA  ON FILIAL.ID_EMPRESA=EMPRESA.ID_EMPRESA
    WHERE VENDA.DATA >= '2022-07-05' AND (REDE_LOJA.DESC_REDE_LOJA ='HERING KIDS'  OR (VENDA.ID_FILIAL='10084' and VENDA.CODIGO_SITE ='004'))
    and exists (select 1 from  view_stores as stores  where FILIAL.CODIGO_FILIAL= stores.STORE_ID))
'''



In [11]:
result_sql = spark.sql(sql)

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8

In [13]:
result_sql.limit(300).toPandas()

CUSTOMER_ID  ORDER_DATE                ORDER_ID ORDER_STORE ORDER_STORE_TYPE ORDER_DELIVERY_TYPE ORDER_STATUS ORDER_PAYMENT_METHOD ORDER_PAYMENT_TYPE ORDER_INSTALLMENTS ORDER_TOTAL __op         timestamp_kafka        MARCA
0                                                       2022-07-06  0105222022070600032701         522                L                                                                                                   89.99    c 2022-07-07 10:00:32.552  HERING KIDS
1    66bcba90d7e8fff22bf8b721c40d17de97fd6a89e0e3a8...  2022-07-05  0108242022070500074650         824                L                                                                                                  119.99    u 2022-07-05 22:11:35.556  HERING KIDS
2    55af1f0b9717010d75bf6a23d23df089df5a0108f86a19...  2022-07-07  0109292022070700083378         929                L                                                                                                  219.98    u 2022-07-07 17:40:44.720  HERING KIDS
3                                                       2022-07-05  0110562022070500036669        1056                L                                                                                                  105.98    c 2022-07-06 13:08:23.218  HERING KIDS
4    e82a43b40ae0c25034e264b36dbd5ec992e7aa3ca1ea00...  2022-07-06  0111342022070600079063        1134                L                                                                                                  145.97    u 2022-07-06 20:40:36.287  HERING KIDS
5                                                       2022-07-06  0111342022070600079072        1134                L                                                                                                    0.00    u 2022-07-06 21:23:44.670  HERING KIDS
6    529186d364ce4946821ee1308d61fd5de58bb7e95ca6af...  2022-07-06  0112362022070600021710        1236                L                                                                                                  139.99    u 2022-07-06 18:05:31.658  HERING KIDS
7                                                       2022-07-07  0113302022070700020031        1330                L                                                                                                   69.99    c 2022-07-07 21:34:47.671  HERING KIDS
8    a7f1eaa5342eb38acc77ef7bed5929b54e69adccfaad18...  2022-07-07  0113322022070700016479        1332                L                                                                                                  109.99    c 2022-07-07 15:32:25.682  HERING KIDS
9    f88fa811a437b5ba7d0370ea09a411ae73916f110f056c...  2022-07-06  0115232022070600010533        1523                L                                                                                                   99.99    u 2022-07-06 14:25:24.645  HERING KIDS
10   a027a7e332a3c5b7643a2444af8d0584bbe16bbe08ff22...  2022-07-06  0115242022070600008966        1524                L                                                                                                  179.99    u 2022-07-06 20:40:36.282  HERING KIDS
11                                                      2022-07-05  0115882022070500004879        1588                L                                                                                                   11.98    c 2022-07-05 16:07:40.166  HERING KIDS
12                                                      2022-07-07  0116172022070700000822        1617                L                                                                                                  319.98    c 2022-07-07 15:42:11.694  HERING KIDS
13                                                      2022-07-05  0116182022070500001827        1618                L                                                                                                  159.98    c 2022-07-05 22:29:43.959  HERING KIDS
14   1b5775a39c89b9c558eeb49849c18cda3f17357648b584..

In [ ]:
sql =  '''
(SELECT
    select 'CUSTOMER_ID|ORDER_DATE|ORDER_ID|ORDER_STORE|ORDER_STORE_TYPE|ORDER_DELIVERY_TYPE|ORDER_STATUS|ORDER_PAYMENT_METHOD|ORDER_PAYMENT_TYPE|ORDER_INSTALLMENTS|ORDER_TOTAL'
    union all
    select 
    rtrim(isnull(CUSTOMER_ID,''))+'|'+	
    rtrim(isnull(ORDER_DATE,''))+'|'+	
    rtrim(isnull(ORDER_ID,''))+'|'+	
    rtrim(isnull(ORDER_STORE,''))+'|'+	
    rtrim(isnull(ORDER_STORE_TYPE,''))+'|'+	
    rtrim(isnull(ORDER_DELIVERY_TYPE,''))+'|'+	
    rtrim(isnull(ORDER_STATUS,''))+'|'+	
    rtrim(isnull(ORDER_PAYMENT_METHOD,''))+'|'+	
    rtrim(isnull(ORDER_PAYMENT_TYPE,''))+'|'+	
    rtrim(isnull(ORDER_INSTALLMENTS,''))+'|'+	
    rtrim(isnull(ORDER_TOTAL,''))
     from (
    SELECT               
      CONVERT(VARCHAR(64), HASHBYTES('SHA2_256',isnull(CASE WHEN (len(REPLACE(REPLACE(VENDA.CPF_CNPJ_CLIENTE,'.',''),'-','')) NOT IN (11,14) or isnumeric(CPF_CNPJ_CLIENTE)=0 ) THEN '000000000000000' ELSE VENDA.CPF_CNPJ_CLIENTE end,'')), 2) AS CUSTOMER_ID,
      CONVERT(DATE,VENDA.DATA) AS ORDER_DATE,
      CONCAT(
              RIGHT(CONCAT('00000', CAST(RTRIM(LTRIM(FILIAL.ID_FILIAL)) AS VARCHAR)),6)
                ,REPLACE(CAST(CAST(VENDA.DATA AS DATE) AS VARCHAR), '-', '')
                ,CAST(RTRIM(LTRIM(VENDA.TICKET)) AS VARCHAR)) AS ORDER_ID,
    RTRIM(LTRIM(FILIAL.CODIGO_FILIAL)) AS ORDER_STORE,
    CASE WHEN FILIAL.CODIGO_FILIAL ='84' THEN 'E' ELSE 'L' END AS ORDER_STORE_TYPE,
    '' AS ORDER_DELIVERY_TYPE,
    '' AS ORDER_STATUS,
    '' AS ORDER_PAYMENT_METHOD,
    '' AS ORDER_PAYMENT_TYPE,
    '' AS ORDER_INSTALLMENTS,
    VENDA.VALOR_VENDA AS ORDER_TOTAL
    FROM DB_MOBILE.DBO.VENDA (NOLOCK) 
    inner join DB_MOBILE.DBO.FILIAL (NOLOCK)  ON FILIAL.ID_FILIAL=VENDA.ID_FILIAL
    INNER JOIN DB_MOBILE.DBO.REDE_LOJA   (NOLOCK) ON REDE_LOJA.ID_REDE_LOJA =FILIAL.ID_REDE_LOJA
    INNER JOIN DB_MOBILE.DBO.EMPRESA  ON FILIAL.ID_EMPRESA=EMPRESA.ID_EMPRESA
    WHERE VENDA.DATA >=@Data
    AND (REDE_LOJA.DESC_REDE_LOJA ='HERING KIDS'  OR (VENDA.ID_FILIAL='10084' and VENDA.CODIGO_SITE ='004')) 
    and exists (select 1 from  DB_MOBILE.DBO.W_OTO_CRM_STORE (NOLOCK) where FILIAL.CODIGO_FILIAL=W_OTO_CRM_STORE.STORE_ID)
    )
'''



In [ ]:
d = datetime.today() - timedelta(days=3)

In [ ]:
her_kids = result_sql.where(result_sql.timestamp_kafka > d)

In [ ]:
her_kids.toPandas()